In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

In [2]:
outer = Rectangle(1,1).Face()
outer.edges.Max(X).name = "r"
outer.edges.Min(X).name = "l"
outer.edges.Min(Y).name = "b"
outer.edges.Max(Y).name = "t"
outer.faces.name = "outer"


mesh = Mesh(OCCGeometry(outer, dim=2).GenerateMesh(maxh=0.2))


In [3]:
inner = MoveTo(0.3,0.5).Rectangle(0.2,0.2).Face()
inner.faces.name = "inner_face"
inner.edges.name = "inner_edge"

In [4]:
geo = Glue([inner, outer])

In [5]:
mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=0.01))
# Draw(mesh)

In [7]:
fes = H1(mesh, order=2, dirichlet="t|b")
u, v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx).Assemble()
gfu = GridFunction(fes)
cf = mesh.MaterialCF({"inner_face":25}, default = 0)

f = LinearForm(cf*v*dx).Assemble()

gfu.Set(cf, definedon = mesh.Materials(".*"))

In [8]:
gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * (f.vec -a.mat * gfu.vec) 

In [9]:
Draw(gfu, mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [10]:
# vecfun = CF((0,1))
n = specialcf.normal(mesh.dim)
grad_u = GridFunction(VectorH1(mesh)) # Why do I have to define VectorH1
grad_u.Set(grad(gfu))

# bottom = -InnerProduct(n, vecfun)
# upper = InnerProduct(n, vecfun)

Draw(grad_u, mesh, vectors=True)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [11]:
Integrate(grad_u*n, mesh.Boundaries('b'))

-0.3999999277496607

In [12]:
Integrate(grad_u* n, mesh.Boundaries('t'))

-0.6000013490148245

### Maybe a sign issue. I dont know if its normal. I guess its because the defined wrt to the outer plane. I expect a posiitve sign because stuff gets into the box

In [50]:

res = f.vec - a.mat * gfu.vec

test_v_top = GridFunction(fes)
test_v_top.Set(1, definedon = mesh.Boundaries("t"))

test_v_bottom = GridFunction(fes)
test_v_bottom.Set(0, definedon = mesh.Boundaries("b"))

res_v_top = InnerProduct(res, test_v_top.vec)


In [51]:
print(res_v_top)

0.6000000000000876


In [52]:
res = f.vec - a.mat * gfu.vec

test_v_top = GridFunction(fes)
test_v_top.Set(0, definedon = mesh.Boundaries("t"))

test_v_bottom = GridFunction(fes)
test_v_bottom.Set(1, definedon = mesh.Boundaries("b"))

res_v_bottom = InnerProduct(res, test_v_bottom.vec)

In [53]:
print(res_v_bottom)

0.40000000000006664
